In [1]:
!pip install optuna -q

In [2]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

import optuna

In [3]:
X_test_mat100 = pd.read_csv('../data/Xte_mat100.csv',sep=' ',header=None).values
X_train_mat100 = pd.read_csv('../data/Xtr_mat100.csv',sep=' ',header=None).values

X_test_ = pd.read_csv('../data/Xte.csv',sep=',',index_col=0)
X_train_ = pd.read_csv('../data/Xtr.csv',sep=',',index_col=0)

y = pd.read_csv('../data/Ytr.csv',sep=',',index_col=0)

train_data = pd.concat([X_train_ , y],axis=1)

In [4]:
print('x_train: {} y_train {}'.format(X_train_mat100.shape,y.shape))
print('x_test: {}'.format(X_test_mat100.shape))


x_train: (2000, 100) y_train (2000, 1)
x_test: (1000, 100)


In [5]:
def get_train_test(X,y,p):
    X = scale(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=p, random_state=42)
    print(X_train.shape,X_test.shape,y_train.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

### Model Test

In [6]:
class logisticregression():
    def __init__(self,train_data,train_labels,lamda=0.2,lr=0.01,decay=10,batch_size=None,epoch=10,print_every = 10):
        dummy_once = np.ones((len(train_data),1))
        self.train_data = np.hstack((dummy_once,train_data))
        self.train_labels = train_labels
        
        self.params = np.zeros((len(self.train_data[0]),1))
        
        self.lr = lr
        self.epoch = epoch
        self.batch_size = batch_size
        self.print_every = print_every
        self._lambda = lamda
        self.decay = decay
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def cost(self,y,y_pred):
        return -np.mean(y*np.log(y_pred)+(1-y)*np.log(1-y_pred))
    
    def gradient(self,y,y_pred,x):
        hassien = np.dot(y_pred.T,(1-y_pred))*np.linalg.pinv(np.dot(x.T,x))
        return np.dot(hassien,np.dot(x.T,(y_pred-y)))+(2*(self._lambda/len(y_pred))*self.params)
#         hassien = np.dot(y_pred.T,(1-y_pred))*np.linalg.inv(np.dot(x.T,x))
#         return np.dot(x.T,(y_pred-y))+(2*(self._lambda/len(y_pred))*self.params)
    
    def train(self):
        for i in range(self.epoch):
            for j in range(len(self.train_labels)//self.batch_size)
                idx = list(np.random.choice(np.arange(len(self.train_labels)),self.batch_size,replace=False))
                data = self.train_data[idx]
                label = self.train_labels[idx]

                y_pred = self.sigmoid(np.dot(data,self.params))
                loss = self.cost(label,y_pred)

                gra = self.gradient(label,y_pred,data)
                self.params -= self.lr*gra

                self.lr *= (1. / (1. + self.decay * i))
            
            if self.print_every:
                if i%self.print_every == 0 or i == self.epoch-1:
                    print('Epoch : {}  Loss: {}'.format(i,loss))
    def predict(self,test_data):
        result = self.sigmoid(np.dot(test_data,self.params[1:])+self.params[0])
        result[result > 0.5 ] = 1
        result[result <= 0.5 ] = 0
        return result
    
    def evaluate(self,test_data,labels):
        accuracy = accuracy_score(self.predict(test_data),labels)
        return accuracy

In [12]:
def cross_validate(x_data,y_data,lr,lamda=0.2,epoch=10,k=5,decay=10):
    if len(x_data)%k != 0:
        print('cant vsplit',len(x_data),' by ',k)
        return
    
    x_data_splitted = np.vsplit(x_data,k)
    y_data_splitted = np.vsplit(y_data,k)
    
    aggrigate_result = []
    for i in range(len(x_data_splitted)):
        train = []
        test = []
        items = [j for j in range(len(x_data_splitted)) if j !=i ]
        x_test = x_data_splitted[i]
        y_test = y_data_splitted[i]
        for item in items:
            if len(train) == 0:
                x_train = x_data_splitted[item]
                y_train = y_data_splitted[item]
            else:
                x_train = np.concatenate((x_train,x_data_splitted[item]), axis=0)
                y_train = np.concatenate((y_train,y_data_splitted[item]), axis=0)
        
        logistic = logisticregression(x_train,y_train,lamda=lamda,lr=lr,decay=decay,epoch=epoch,print_every=100)
        logistic.train()
        
        result = logistic.evaluate(x_test,y_test)
        aggrigate_result.append(result)
        
    value = sum(aggrigate_result)/len(aggrigate_result)
    return value

# Optimize

In [13]:
def getKmers(sequence, size=6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

train_data['words'] = train_data.seq.apply(lambda x: ' '.join(getKmers(x)))
X_test_['words'] = X_test_.seq.apply(lambda x: ' '.join(getKmers(x)))
train_data.head(2)

,seq,Bound,words
Id,,,
0,GAGGGGCTGGGGAGGGGGCTGGCCCAGAGGCACCAGACTCTGCAGA...,1,gagggg aggggc ggggct gggctg ggctgg gctggg ctgg...
1,CGGCCTGGGGGCCACATGTGAGTGCTTACCTGTGTGGGGATGAGGG...,0,cggcct ggcctg gcctgg cctggg ctgggg tggggg gggg...


In [14]:
from sklearn.feature_extraction.text import CountVectorizer


data = pd.DataFrame(pd.concat([train_data.words,X_test_.words],axis=0))

train_text = data.words.values

cv = CountVectorizer(ngram_range=(2,2),max_features=1500)
X = cv.fit_transform(train_text)
X = X.todense()


X.shape

(3000, 1500)

In [ ]:
cross_validate(np.array(X)[:2000,:],y.values,k=5,lr=0.001,lamda=0.0003,epoch=10)

Epoch : 0  Loss: 0.6931471805599453


In [71]:
def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    lamda = trial.suggest_loguniform('lamda', 1e-5, 10)
    k =  trial.suggest_categorical('k', [4,5,8,10])
    epoch =  trial.suggest_int('epoch', 90, 100)
    decay = trial.suggest_int('decay', 3, 10)
    return cross_validate(np.array(X)[:2000,:], y,lr=lr,lamda=lamda,k=k,epoch=epoch,decay=decay)
# cross_validate(X_preprocess, y.reshape(-1,1),lr=0.001,epoch=200)

import optuna

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=objective, n_trials=100,show_progress_bar=True)

/home/aimsgh/anaconda3/lib/python3.7/site-packages/optuna/_experimental.py:90: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  ExperimentalWarning,


[229, 157, 397, 212, 224, 15, 40, 331, 102, 8, 26, 248, 69, 190, 200, 77, 219, 267, 312, 76, 383, 225, 360, 140, 160, 205, 117, 108, 397, 202, 0, 31, 177, 57, 7, 62, 30, 271, 8, 61, 321, 80, 192, 118, 198, 328, 144, 288, 232, 312, 370, 49, 92, 28, 284, 165, 26, 363, 234, 122, 162, 22, 140, 233]
[390, 213, 188, 318, 318, 359, 180, 235, 160, 143, 176, 181, 284, 67, 193, 237, 335, 112, 77, 294, 261, 375, 5, 292, 321, 353, 166, 220, 110, 330, 229, 28, 203, 17, 84, 206, 143, 267, 251, 128, 328, 105, 137, 184, 8, 177, 338, 220, 380, 81, 224, 221, 58, 371, 48, 123, 298, 44, 85, 141, 144, 164, 274, 75]
[85, 205, 301, 226, 257, 201, 384, 243, 120, 354, 226, 336, 378, 320, 201, 70, 150, 197, 225, 133, 103, 390, 243, 221, 40, 35, 381, 75, 382, 230, 328, 335, 215, 343, 74, 182, 160, 77, 39, 228, 29, 164, 103, 178, 134, 290, 359, 78, 195, 119, 389, 152, 336, 100, 11, 77, 159, 333, 142, 284, 175, 352, 109, 44]
[279, 16, 157, 121, 75, 325, 300, 51, 214, 31, 336, 149, 75, 218, 127, 103, 342, 381, 69, 

[205, 381, 328, 8, 180, 351, 393, 306, 11, 238, 321, 268, 356, 240, 340, 398, 149, 248, 339, 260, 317, 240, 39, 297, 280, 293, 204, 191, 336, 201, 322, 91, 68, 4, 165, 70, 251, 224, 284, 114, 60, 381, 43, 91, 164, 397, 19, 125, 349, 307, 222, 386, 56, 9, 210, 167, 165, 344, 240, 32, 41, 47, 147, 154]
[265, 19, 2, 197, 136, 173, 273, 219, 244, 206, 118, 116, 71, 182, 30, 166, 343, 141, 382, 152, 293, 209, 141, 393, 130, 230, 11, 107, 42, 221, 286, 184, 44, 309, 367, 148, 329, 214, 137, 218, 188, 309, 297, 67, 253, 369, 64, 114, 0, 19, 393, 286, 333, 83, 66, 227, 273, 228, 145, 4, 337, 60, 102, 286]
[218, 163, 94, 339, 151, 8, 352, 310, 323, 138, 135, 58, 214, 23, 280, 132, 247, 184, 165, 188, 255, 80, 253, 128, 12, 78, 360, 273, 149, 47, 279, 295, 214, 355, 368, 139, 119, 285, 128, 242, 146, 46, 39, 169, 137, 136, 161, 200, 377, 382, 126, 173, 26, 379, 187, 243, 17, 338, 154, 240, 229, 81, 192, 154]
[258, 153, 147, 391, 101, 362, 151, 234, 170, 319, 379, 159, 342, 309, 56, 180, 107, 10,

[254, 175, 211, 392, 51, 392, 94, 284, 13, 198, 257, 361, 10, 274, 120, 317, 265, 58, 218, 361, 355, 311, 128, 114, 144, 25, 371, 210, 381, 115, 195, 272, 62, 153, 286, 215, 304, 283, 135, 326, 166, 258, 197, 208, 185, 121, 368, 237, 192, 214, 139, 107, 140, 181, 254, 194, 72, 35, 343, 132, 343, 349, 348, 382]
[172, 229, 336, 292, 103, 158, 145, 382, 91, 371, 129, 61, 254, 169, 313, 237, 363, 355, 89, 188, 252, 106, 154, 98, 146, 145, 241, 154, 188, 6, 118, 158, 385, 248, 91, 357, 389, 211, 97, 195, 70, 327, 395, 72, 257, 181, 204, 10, 154, 240, 133, 390, 141, 59, 269, 39, 340, 221, 88, 233, 119, 211, 325, 173]
[392, 154, 250, 84, 22, 69, 124, 289, 185, 148, 326, 330, 196, 298, 336, 101, 232, 315, 294, 397, 387, 139, 116, 62, 240, 9, 355, 66, 329, 192, 238, 97, 215, 220, 329, 32, 130, 323, 196, 20, 374, 147, 152, 36, 275, 358, 48, 325, 348, 122, 200, 245, 241, 46, 339, 260, 36, 129, 174, 121, 145, 141, 381, 122]
[309, 282, 159, 172, 238, 93, 180, 354, 278, 379, 273, 163, 145, 138, 121,

[160, 243, 209, 57, 367, 269, 186, 221, 287, 161, 259, 357, 277, 0, 55, 63, 182, 184, 205, 153, 35, 24, 262, 297, 307, 197, 199, 290, 67, 77, 84, 329, 28, 104, 30, 43, 141, 274, 23, 236, 381, 272, 32, 312, 81, 109, 256, 110, 53, 6, 301, 156, 241, 184, 9, 314, 223, 82, 237, 134, 48, 320, 334, 230]
[311, 254, 357, 362, 376, 86, 242, 159, 257, 56, 154, 325, 341, 7, 26, 373, 237, 61, 298, 46, 74, 165, 52, 24, 285, 58, 238, 397, 379, 351, 23, 392, 262, 13, 39, 167, 281, 234, 286, 15, 42, 325, 221, 1, 305, 65, 173, 98, 364, 369, 105, 124, 269, 322, 266, 49, 377, 314, 390, 337, 79, 191, 181, 371]
[4, 287, 121, 283, 270, 362, 108, 318, 251, 245, 198, 327, 274, 345, 239, 63, 343, 234, 397, 306, 73, 313, 16, 56, 373, 298, 14, 238, 7, 291, 161, 186, 33, 233, 327, 370, 42, 267, 219, 320, 60, 112, 194, 372, 265, 63, 127, 183, 269, 89, 396, 357, 31, 199, 54, 31, 23, 106, 226, 13, 349, 167, 243, 99]
[259, 299, 78, 291, 243, 236, 244, 303, 53, 134, 60, 390, 294, 159, 173, 44, 171, 40, 126, 342, 315, 3

...

[487, 419, 151, 430, 140, 348, 88, 167, 251, 82, 351, 110, 497, 21, 273, 204, 264, 443, 141, 173, 437, 23, 425, 143, 440, 76, 392, 209, 485, 231, 254, 15, 240, 184, 162, 50, 112, 399, 278, 402, 73, 91, 498, 105, 262, 491, 399, 412, 416, 127, 250, 263, 281, 495, 311, 476, 257, 214, 244, 246, 363, 160, 351, 216]
[311, 79, 70, 343, 301, 372, 91, 60, 176, 167, 247, 433, 490, 148, 79, 336, 491, 87, 153, 60, 315, 426, 376, 225, 81, 31, 248, 405, 301, 272, 296, 20, 420, 415, 482, 87, 121, 387, 32, 418, 215, 372, 222, 31, 60, 285, 160, 116, 453, 126, 308, 181, 203, 135, 326, 469, 10, 1, 427, 438, 150, 417, 451, 403]
[464, 427, 155, 183, 364, 158, 292, 487, 124, 94, 313, 279, 25, 6, 427, 308, 427, 66, 472, 428, 346, 126, 355, 303, 482, 53, 407, 308, 487, 202, 183, 111, 110, 268, 38, 71, 136, 407, 362, 460, 173, 287, 185, 428, 418, 498, 438, 193, 107, 296, 323, 126, 494, 24, 489, 76, 17, 348, 406, 451, 255, 264, 244, 469]
[181, 38, 89, 458, 470, 371, 203, 319, 6, 285, 481, 15, 254, 155, 189, 58,

[427, 236, 318, 314, 440, 295, 497, 191, 322, 248, 222, 400, 183, 263, 399, 476, 217, 258, 337, 378, 305, 462, 303, 212, 487, 370, 241, 181, 234, 17, 203, 387, 393, 237, 300, 313, 322, 344, 289, 331, 71, 8, 291, 19, 56, 263, 404, 95, 101, 453, 359, 107, 345, 183, 213, 292, 226, 102, 89, 114, 124, 155, 85, 477]
[106, 421, 332, 402, 434, 423, 0, 127, 208, 396, 158, 112, 405, 242, 102, 335, 6, 29, 241, 238, 201, 21, 42, 181, 121, 215, 259, 485, 389, 364, 263, 328, 78, 370, 105, 46, 417, 219, 329, 107, 299, 128, 155, 324, 462, 484, 52, 22, 389, 367, 86, 338, 219, 406, 376, 55, 156, 177, 35, 353, 113, 279, 39, 280]
[194, 498, 446, 409, 291, 325, 445, 332, 163, 203, 224, 167, 114, 481, 158, 218, 371, 19, 269, 336, 15, 403, 369, 287, 176, 476, 273, 211, 270, 192, 80, 258, 140, 186, 334, 452, 27, 37, 375, 460, 144, 140, 346, 295, 399, 334, 112, 114, 324, 418, 156, 88, 169, 93, 36, 486, 478, 145, 339, 46, 357, 39, 94, 475]
[270, 443, 30, 212, 423, 140, 290, 436, 204, 384, 320, 374, 356, 346, 10

[75, 342, 255, 23, 305, 397, 291, 459, 146, 446, 443, 288, 85, 484, 429, 334, 263, 182, 373, 499, 328, 357, 76, 312, 87, 138, 290, 255, 338, 318, 365, 24, 73, 238, 68, 146, 27, 454, 388, 63, 53, 173, 458, 216, 207, 261, 101, 301, 100, 176, 189, 129, 437, 109, 236, 463, 40, 400, 81, 196, 328, 110, 406, 309]
[445, 208, 377, 215, 288, 49, 313, 125, 260, 427, 262, 462, 484, 292, 11, 272, 288, 31, 91, 321, 454, 361, 378, 366, 428, 322, 427, 269, 279, 486, 187, 493, 84, 179, 84, 32, 50, 469, 447, 27, 128, 468, 123, 130, 205, 347, 59, 421, 297, 174, 448, 224, 309, 220, 320, 177, 13, 134, 82, 150, 286, 355, 14, 35]
[296, 394, 169, 325, 437, 207, 446, 98, 420, 252, 440, 440, 166, 230, 450, 138, 3, 7, 336, 90, 305, 368, 52, 100, 206, 46, 289, 95, 289, 346, 295, 95, 364, 25, 251, 300, 293, 23, 235, 216, 128, 155, 285, 78, 463, 465, 155, 375, 101, 32, 226, 36, 3, 371, 131, 401, 369, 140, 11, 76, 405, 390, 35, 429]
[6, 414, 296, 127, 257, 268, 452, 369, 336, 139, 377, 499, 403, 277, 464, 207, 9, 30

[46, 313, 265, 445, 38, 457, 425, 332, 360, 29, 287, 43, 463, 4, 434, 97, 315, 352, 275, 443, 114, 103, 202, 364, 277, 56, 20, 58, 455, 124, 451, 91, 236, 77, 195, 427, 304, 161, 258, 453, 493, 340, 114, 456, 323, 490, 436, 234, 2, 12, 410, 40, 64, 12, 271, 108, 284, 327, 184, 92, 279, 7, 246, 10]
[235, 460, 271, 64, 333, 67, 82, 417, 48, 351, 207, 412, 60, 257, 12, 73, 257, 322, 256, 369, 118, 422, 3, 73, 368, 216, 344, 54, 349, 39, 481, 29, 284, 23, 279, 344, 4, 357, 498, 96, 262, 495, 84, 373, 424, 260, 191, 191, 59, 156, 330, 464, 176, 466, 496, 320, 307, 51, 128, 131, 402, 294, 84, 240]
[250, 488, 481, 489, 351, 321, 45, 257, 451, 163, 241, 375, 383, 155, 174, 352, 495, 283, 266, 119, 413, 5, 432, 355, 8, 25, 243, 345, 406, 255, 383, 403, 498, 16, 26, 486, 48, 297, 306, 433, 455, 467, 428, 445, 261, 41, 439, 253, 244, 190, 240, 146, 389, 9, 74, 122, 453, 127, 10, 146, 463, 108, 403, 239]
[408, 120, 278, 191, 310, 37, 496, 3, 61, 461, 148, 415, 121, 483, 439, 415, 127, 455, 375, 27

KeyboardInterrupt: 

In [90]:
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.sort_values(by=['value'])

,number,value,duration,params_decay,params_epoch,params_k,params_lamda,params_lr
8,8,0.5000,00:00:01.543956,10,805,8,0.000134,0.094504
0,0,0.5505,00:00:00.687026,10,950,4,0.060581,0.025797
30,30,0.5980,00:00:01.211547,8,675,8,0.000034,0.001577
86,86,0.6010,00:00:01.273208,4,514,8,0.000736,0.001310
4,4,0.6090,00:00:00.670439,6,536,4,0.002069,0.017855
...,...,...,...,...,...,...,...,...
42,42,0.7005,00:00:00.880267,5,610,5,0.000116,0.001898
87,87,0.7015,00:00:00.574556,6,630,5,0.000028,0.001995
61,61,0.7015,00:00:00.858273,5,607,5,0.000043,0.001978
41,41,0.7020,00:00:00.897417,5,606,5,0.000138,0.001953


In [50]:
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.sort_values(by=['value'])

(3000, 1)

In [48]:
#DNA sequence as a “language”, known as k-mer counting
def getKmers(sequence, size):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]
def get_n_grams(data1,n):
    X_train = []
    X_test = []

    cv = CountVectorizer(analyzer='char',ngram_range=(n,n))
    for i in data1:
        sentence = ' '.join(getKmers(i[0], size=n))
        X_train.append(sentence)
        
    X_cocat = X_train
    X = cv.fit_transform(X_cocat).toarray()
    return X

X_preprocess = get_n_grams(X_train_,7)

# def objective(trial):
#     lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
#     lamda = trial.suggest_loguniform('lamda', 0.01, 0.5)
#     k =  trial.suggest_categorical('k', [4,5,8,10])
#     epoch =  trial.suggest_int('epoch', 10, 20)
#     decay = trial.suggest_int('decay', 3, 10)
#     return cross_validate(X_preprocess[:2000,:], y,lr=lr,lamda=lamda,k=k,epoch=epoch,decay=decay)

cross_validate(X_preprocess[:2000], y,0.001,20,k=5,epoch=10,decay=10)

# import optuna

# sampler = optuna.samplers.TPESampler()
# study = optuna.create_study(sampler=sampler, direction='maximize')
# study.optimize(func=objective, n_trials=100,show_progress_bar=True)

0.924

In [13]:
# df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
# df.sort_values(by=['value'])

In [91]:
# # Count Vectorizer 
# def get_count_grams(data1,n):
#     cv = CountVectorizer(analyzer='char',ngram_range=(n,n))
#     X = cv.fit_transform(data1).toarray()
#     return X

# X_preprocess = get_n_grams(X_train_.flatten(),8)

# # def objective(trial):
# #     lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
# #     lamda = trial.suggest_loguniform('lamda', 0.01, 0.5)
# #     k =  trial.suggest_categorical('k', [4,5,8,10])
# #     epoch =  trial.suggest_int('epoch', 10, 20)
# #     decay = trial.suggest_int('decay', 3, 10)
# #     return cross_validate(X_preprocess, y,lr=lr,lamda=lamda,k=k,epoch=epoch,decay=decay)

# cross_validate(X_preprocess[:2000], y,0.0001,20,k=5,epoch=10,decay=10)

# # import optuna

# # sampler = optuna.samplers.TPESampler()
# # study = optuna.create_study(sampler=sampler, direction='maximize')
# # study.optimize(func=objective, n_trials=200,show_progress_bar=True)

In [201]:
# df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
# df.sort_values(by=['value'])

In [202]:
# cross_validate(X_preprocess, y,lr=0.004433,lamda=0.432127,k=4,epoch=16,decay=4)

In [66]:
# Count Vectorizer 
def get_tf_idf_grams(data1,n):
    cv = TfidfVectorizer(analyzer='char',ngram_range=(n,n))
    X = cv.fit_transform(data1).toarray()
    return X

X_preprocess = get_tf_idf_grams(X_train_.flatten(),8)

# def objective(trial):
#     lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
#     lamda = trial.suggest_loguniform('lamda', 0.01, 0.5)
#     k =  trial.suggest_categorical('k', [4,5,8,10])
#     epoch =  trial.suggest_int('epoch', 10, 20)
#     decay = trial.suggest_int('decay', 3, 10)
#     return cross_validate(X_preprocess, y,lr=lr,lamda=lamda,k=k,epoch=epoch,decay=decay)

cross_validate(X_preprocess[:2000], y,0.001,20,k=5,epoch=10,decay=10)

# import optuna

# sampler = optuna.samplers.TPESampler()
# study = optuna.create_study(sampler=sampler, direction='maximize')
# study.optimize(func=objective, n_trials=100,show_progress_bar=True)

0.5

In [213]:
# df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
# df.sort_values(by=['value'])

# After testing all possible dataset preprocessing type now lets stick to one

In [ ]:
X_train, X_test, y_train, y_test = get_train_test(X_preprocess[:2000],y,0.3)


In [112]:
# X_preprocess = get_n_grams(X_train_,8)
# X_preprocess.shape

# print(cross_validate(X_preprocess[:2000,:], y,lr=0.001,lamda=15,k=4,epoch=16,decay=10))

X_train, X_test, y_train, y_test = get_train_test(XXX,y,0.01)

# y,0.001,15,k=5,epoch=10,decay=10)

logistic = logisticregression(X_train,y_train,lamda=0.000168,lr=0.029734,decay=4,epoch=700,print_every=None)
logistic.train()
        
print(logistic.evaluate(X_train,y_train))
print(logistic.evaluate(X_test,y_test))
cross_validate(XXX, y,lamda=0.000168,lr=0.029734,decay=4,epoch=700)

(1980, 50) (20, 50) (1980, 1) (20, 1)


/home/aimsgh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log
/home/aimsgh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in multiply


0.5818181818181818
0.65


[0.7075, 0.7075, 0.7075, 0.7075, 0.69]

In [235]:
X_preprocess = get_count_grams(np.vstack((X_train_,X_test_)).flatten(),6)
X_preprocess.shape

cross_validate(X_preprocess[:2000,:], y,lr=0.004433,lamda=0.432127,k=4,epoch=16,decay=4)

C_count_6 = X_preprocess[2000:,:]

X_train, X_test, y_train, y_test = get_train_test(X_preprocess[:2000,:],y,0.3)


logistic_count6 = logisticregression(X_train,y_train,lamda=0.455265,epoch=10,print_every=1,lr=0.000407,decay=11)
logistic_count6.train()
        
print(logistic_count6.evaluate(X_train,y_train))
print(logistic_count6.evaluate(X_test,y_test))

(1400, 4096) (600, 4096) (1400, 1) (600, 1)
Epoch : 0  Loss: 0.6931471805599454
Epoch : 1  Loss: 0.369619629360289
Epoch : 2  Loss: 0.24093457515641262
Epoch : 3  Loss: 0.23515736618853034
Epoch : 4  Loss: 0.2349297795537864
Epoch : 5  Loss: 0.23492310904273742
Epoch : 6  Loss: 0.23492296082415293
Epoch : 7  Loss: 0.23492295817739842
Epoch : 8  Loss: 0.23492295813789463
Epoch : 9  Loss: 0.2349229581373882
0.9664285714285714
0.6433333333333333


In [236]:
X_preprocess = get_count_grams(np.vstack((X_train_,X_test_)).flatten(),4)
X_preprocess.shape

print(cross_validate(X_preprocess[:2000,:], y,lr=0.004433,lamda=0.432127,k=4,epoch=16,decay=7))

C_count_4 = X_preprocess[2000:,:]

X_train, X_test, y_train, y_test = get_train_test(X_preprocess[:2000,:],y,0.3)


logistic_count4 = logisticregression(X_train,y_train,lamda=0.455265,epoch=10,print_every=1,lr=0.000407,decay=11)
logistic_count4.train()
        
print(logistic_count4.evaluate(X_train,y_train))
print(logistic_count4.evaluate(X_test,y_test))

(1400, 256) (600, 256) (1400, 1) (600, 1)
Epoch : 0  Loss: 0.6931471805599454
Epoch : 1  Loss: 0.6286425932017687
Epoch : 2  Loss: 0.6042153578172307
Epoch : 3  Loss: 0.6027152714606171
Epoch : 4  Loss: 0.6026532220481413
Epoch : 5  Loss: 0.6026514005159469
Epoch : 6  Loss: 0.6026513600396904
Epoch : 7  Loss: 0.6026513593169011
Epoch : 8  Loss: 0.6026513593061132
Epoch : 9  Loss: 0.6026513593059748
0.6907142857142857
0.59


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning:

invalid value encountered in multiply



In [237]:
X_preprocess = get_tf_idf_grams(np.vstack((X_train_,X_test_)).flatten(),4)
X_preprocess.shape

cross_validate(X_preprocess[:2000,:], y,lr=0.004433,lamda=0.432127,k=4,epoch=16,decay=4)

C_tf_4 = X_preprocess[2000:,:]


X_train, X_test, y_train, y_test = get_train_test(X_preprocess[:2000,:],y,0.3)


logistic_tf4 = logisticregression(X_train,y_train,lamda=0.455265,epoch=10,print_every=1,lr=0.000407,decay=11)
logistic_tf4.train()
        
print(logistic_tf4.evaluate(X_train,y_train))
print(logistic_tf4.evaluate(X_test,y_test))

(1400, 256) (600, 256) (1400, 1) (600, 1)
Epoch : 0  Loss: 0.6931471805599454
Epoch : 1  Loss: 0.6260173199159424
Epoch : 2  Loss: 0.6002614260404124
Epoch : 3  Loss: 0.598726394674755
Epoch : 4  Loss: 0.5986622185693936
Epoch : 5  Loss: 0.5986603339307318
Epoch : 6  Loss: 0.5986602920517511
Epoch : 7  Loss: 0.598660291303913
Epoch : 8  Loss: 0.5986602912927511
Epoch : 9  Loss: 0.5986602912926081
0.7028571428571428
0.5966666666666667


In [238]:
X_preprocess = get_tf_idf_grams(np.vstack((X_train_,X_test_)).flatten(),6)
X_preprocess.shape

cross_validate(X_preprocess[:2000,:], y,lr=0.004433,lamda=0.432127,k=4,epoch=16,decay=7)

C_tf_6 = X_preprocess[2000:,:]

X_train, X_test, y_train, y_test = get_train_test(X_preprocess[:2000,:],y,0.3)


logistic_tf6 = logisticregression(X_train,y_train,lamda=0.455265,epoch=10,print_every=1,lr=0.000407,decay=11)
logistic_tf6.train()
        
print(logistic_tf6.evaluate(X_train,y_train))
print(logistic_tf6.evaluate(X_test,y_test))

(1400, 4096) (600, 4096) (1400, 1) (600, 1)
Epoch : 0  Loss: 0.6931471805599454
Epoch : 1  Loss: 0.3746954683721624
Epoch : 2  Loss: 0.23994925474338094
Epoch : 3  Loss: 0.23422424550847226
Epoch : 4  Loss: 0.23399751150543982
Epoch : 5  Loss: 0.23399086485544413
Epoch : 6  Loss: 0.23399071716632397
Epoch : 7  Loss: 0.2339907145290239
Epoch : 8  Loss: 0.2339907144896612
Epoch : 9  Loss: 0.23399071448915654
0.9635714285714285
0.6433333333333333


In [15]:
X_test_final = scale(X_preprocess[2000:])

In [17]:
from collections import Counter


sumbission = []
for i in range(len(X_test_final)):
    r1 = logistic.predict(X_test_final[i])
#     r2 = logistic_count6.predict(C_count_6[i])
#     r3 = logistic_tf4.predict(C_tf_4[i])
#     r4 = logistic_tf6.predict(C_tf_6[i])
    
    
#     votes = [r1[0],r2[0],r3[0],r4[0]]
    
#     print(Counter(votes))
#     print(Counter(votes).most_common(1)[0][0])
    
#     break
#     sumbission.append([i,int(Counter(votes).most_common(1)[0][0])])
    
    sumbission.append([i,int(r1)])

In [20]:
# sumbission
df = pd.DataFrame(sumbission)
df.columns = ['Id','Bound']
df.to_csv('cv_64.9.csv',index=False)

In [19]:
df.head(50)

,Id,Bound
0,0,1
1,1,1
2,2,0
3,3,0
4,4,1
5,5,1
6,6,1
7,7,1
8,8,1
9,9,1
